In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
cd gdrive/MyDrive/MachineLearning/HandsOnMachineLearning/spam-classifier

/content/gdrive/MyDrive/MachineLearning/HandsOnMachineLearning/spam-classifier


In [3]:
import os


HAM_DIR = "easy_ham"
SPAM_DIR = "spam"
ham_filenames = [name for name in sorted(os.listdir(HAM_DIR)) if len(name) > 20]
spam_filenames = [name for name in sorted(os.listdir(SPAM_DIR)) if len(name) > 20]

print(len(ham_filenames))
len((spam_filenames))

2500


500

In [ ]:
import email
import email.policy

def load_email(is_spam, filename):
    directory = "spam" if is_spam else "easy_ham"

    with open(os.path.join(directory, filename), "rb") as f:
        email_obj = email.parser.BytesParser(policy=email.policy.default).parse(f)

        if email_obj.is_multipart():
            content = ''
            msgs = email_obj.get_payload()
            for msg in msgs:
                content_type = msg.get_content_type()
                if content_type == 'text/plain' or content_type == 'text/html':
                    content += msg.get_content()
        else:
            content = email_obj.get_payload()

        content = content.strip()
        return content

ham_emails = [load_email(is_spam=False, filename=name) for name in ham_filenames]
spam_emails = [load_email(is_spam=True, filename=name) for name in spam_filenames]


In [ ]:
# TODO create dataframe from lists, one column text, second column spam (0/1)

In [52]:
import re
import string
import nltk
#nltk.download('popular')


test = """ On Mon, 2 Sep 2002, Reza B'Far (eBuilt) wrote:

> With the increasing prevalence of web services (not that they are always a
> good thing), I doubt that parsing XML will be something that will remain at
> the Java application layer for long... Recent threads here on Fork
> indicating the move towards hardware parsing or this code even become part
> of the native implementation of Java on various platforms...

OK, so, you get the XML toss it through hardware and turn it back into a
struct/object whatever you call your binary data. I agree this is the way
things will go, as XML parsing has just too much overhead to survive in the
application layer. 111111111 So 123 wh1y turn it into XML in the first place? Becasue it
gives geeks something to do and sells XML hardware accelerators and way more
CPUs?

Is there anyone out there actually doing anything new that actually IMPROVES
things anymore, or are they all too scared of the fact that improvements put
people out of work and cut #1 is the creators... """


sno = nltk.stem.SnowballStemmer('english')


def transform_text(text, lowercase=False, remove_header=False,
                   remove_punctuation=False, replace_url=False,
                   replace_numbers=False, stemming=False):
    transformed_text = text

    if lowercase:
        transformed_text = transformed_text.lower()

    if remove_header:
        transformed_text = re.sub(r'^.*?^\s*$', '', transformed_text, count=1, flags=re.MULTILINE | re.DOTALL)

    if remove_punctuation:
        transformed_text = transformed_text.strip(string.punctuation)

    if replace_url:
        transformed_text = re.sub(r"(?i)\b((?:[a-z][\w-]+:(?:/{1,3}|[a-z0-9%])|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))",
                                  'URL', transformed_text, count=0, flags=re.MULTILINE)
    if replace_numbers:
        transformed_text = re.sub(r'\d+', 'NUMBER', transformed_text, count=0)

    if stemming:
        split_text = nltk.word_tokenize(transformed_text)
        transformed_text = ' '.join([sno.stem(word) for word in split_text])

    return transformed_text


print(transform_text(test, replace_numbers=True, stemming=True))

on mon , number sep number , reza b'far ( ebuilt ) wrote : > with the increas preval of web servic ( not that they are alway a > good thing ) , i doubt that pars xml will be someth that will remain at > the java applic layer for long ... recent thread here on fork > indic the move toward hardwar pars or this code even becom part > of the nativ implement of java on various platform ... ok , so , you get the xml toss it through hardwar and turn it back into a struct/object whatev you call your binari data . i agre this is the way thing will go , as xml pars has just too much overhead to surviv in the applic layer . number so number whnumberi turn it into xml in the first place ? becasu it give geek someth to do and sell xml hardwar acceler and way more cpus ? is there anyon out there actual do anyth new that actual improv thing anymor , or are they all too scare of the fact that improv put peopl out of work and cut # number is the creator ...


In [ ]:
# print(ham_emails[1].get_content().strip())
# print(spam_emails[6].get_content().strip())
# print(type(ham_emails[1]))
print(ham_emails[1].get_content().strip())

In [ ]:
from sklearn.pipeline import Pipeline

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer


vectorizer = CountVectorizer()